In [1]:
import pandas as pd
import numpy as np
import os
from math import *

In [2]:
#一次读取多个文件的大小
def get_dir_size(dir): #1M=1048576字节 200M=209715200字节
    size = 0 #初始化size
    for root, dirs, files in os.walk(dir): #遍历文件夹内每一个子文件
        size += sum([os.path.getsize(root + '/'+ name) for name in files]) #计算总大小
    return size

In [3]:
#判断文件是否大于200M，即是否完成收敛
def get_origin_data (i,dirpath):
    size = get_dir_size(dirpath) #引用函数获取文件大小
    print (i,size) #输出每个文件夹的大小
    if size >= 209715200: #判断文件夹是否大于200M，大于则输出1用于判断
        a = 1
    else:
        a = 0
    return a #返回判断值和文件夹路径

In [4]:
#读取文件并放入pandas进行处理
def read_file (file):
    data = pd.read_csv(file)
    return data

In [5]:
#获取CONTCAR文件的晶格常数并按列放入dataframe
def get_lattice(data,df,i):
    b = [] #初始化一个列表用于放入三行晶格常数
    for i1 in [1,2,3]: #遍历三行晶格常数
        a = data.iloc[i1,0] #读取对应行标的晶格常数，str格式
        a = a.split(' ') #str格式带有多个空格，用空格作为判断标准分成列表
        b = b+a #合成一个列表
    while '' in b: #把空格项删除
        b.remove('')
    df['{}'.format(i)] = b #把列表的晶格常数按列放入dataframe
    return df,b #返回含有晶格常数的dataframe，并返回一个含有abc的列表用于计算表面内应力

In [6]:
# 把含有晶格常数的dataframe对每一行求平均值，求得平均晶格常数
def get_mean_lattice(df): 
    df = df.astype('float') #把dataframe所有元素转换成float类型
    df['均值']=df.mean(axis=1) #按行求平均值，列名为均值
    mean = df.iloc[:,-1] #把平均值放入mean
    print ('均值为：{}'.format(mean)) #输出均值
    return mean #返回mean

In [7]:
# 把mean中的晶格常数分为a,b,c三行，用于放入CONTCAR文件中
def get_mean_abc(mean):
    a = str(mean.iloc[0])+' '+str(mean.iloc[1])+' '+str(mean.iloc[2])
    b = str(mean.iloc[3])+' '+str(mean.iloc[4])+' '+str(mean.iloc[5])
    c = str(mean.iloc[6])+' '+str(mean.iloc[7])+' '+str(mean.iloc[8])
    return a,b,c #返回三行晶格常数均值

In [8]:
# 用三行晶格常数均值替换掉原CONTACR文件data的晶格常数
def get_train_contcar(a,b,c,data):
    data.loc[1,:]=a
    data.loc[2,:]=b
    data.loc[3,:]=c
    return data #返回替换后的CONTCAR

In [9]:
# 把新的CONTCAR数据data按path路径和文件名i写入新的CONTCAR文件
def write_contcar(i,data,path):
    data.to_csv(path + '/CONTCAR{}'.format(i),index=False)

In [10]:
# 按照规定的path1和path2路径创建放置CONTCAR和cif文件的文件夹
def makedir(path1,path2,path3,path4):
        if os.path.exists(path1) == 0: #判断路径是否存在对应文件夹，有则略过，无则创建
            os.mkdir(path1)
        if os.path.exists(path2) == 0:
            os.mkdir(path2)
        if os.path.exists(path3) == 0:
            os.mkdir(path3)
        if os.path.exists(path4) == 0:
            os.mkdir(path4)

In [11]:
# 转换原始cif并提取abc用于表面应力计算
def get_abc(file,path4,i2):
    os.system('ase convert {} {}/{}.cif'.format(file,path4,i2))
    data = pd.read_csv('{}/{}.cif'.format(path4,i2),error_bad_lines = False)
    ls = []
    for i in range(2,5):
        l = data.iloc[i,0] #读取对应行标的晶格常数，str格式
        l = l.split(' ') #str格式带有多个空格，用空格作为判断标准分成列表
        ls = ls+l #合成一个列表
    while '' in ls: #把空格项删除
        ls.remove('')
    a = ls[1]
    b = ls[3]
    c = ls[5]
    return a,b,c

In [12]:
# 计算表面应力
def get_strain(a,b,c):
    l = sqrt((pow(a,2)+pow(b,2)))
    m = sqrt((pow(a,2)+pow(c,2)))
    n = sqrt((pow(b,2)+pow(c,2)))
    p = (l + m + n)/2
    s = sqrt(p * (p-l) * (p-m) * (p-n))
    return s

In [13]:
# 计算表面应力差值（与PtCu3作比较）
def get_strain_via_ptcu3(a,b,c):
    s_ptcu3 = get_strain(7.45216 ,7.45215 , 7.45216)
    suface_strain = (get_strain(a,b,c) - s_ptcu3) / s_ptcu3
    return suface_strain

In [14]:
# 以下为读取E0函数

In [15]:
# 读取OSZICAR文件，并取最后一行
def get_oszicar(file_OSZICAR):
    with open(file_OSZICAR, 'r', encoding='utf-8') as f:  # 打开文件
        lines = f.readlines()  # 读取所有行
        last_line = lines[-1]  # 取最后一行
    return last_line #返回最后一行

In [16]:
def get_train_atom(data):
    atom = data.iloc[4,0]
    counts = data.iloc[5,0]
    atom = atom.split(' ')
    counts = counts.split(' ')
    while '' in atom: #把空格项删除
        atom.remove('')
    while '' in counts: #把空格项删除
        counts.remove('')
    return atom,counts

In [17]:
# 在最后一行的文本中提取出E0的值
def get_e0(last_line , atom , counts , data_energy):
    last_line = last_line.split(' ') #按文本中的空格分割出列表
# 遍历列表，把'E0='后面的数值取出，此处特定E0数值为负数，如果为正数则需要加一个空值判断
    for i in range (len(last_line)): 
        if last_line[i] == 'E0=':
            E0 = float(last_line[i+1])
    for i in range(len(atom)):
        E0 = E0 - data_energy.loc[atom[i],0] * int(counts[i])
    return E0 #返回E0数值

In [18]:
# 把文件号和E0写入CSV，创建ML命令CSV文件
def write_csv_e0(i,E0,path2):
    csv_data = [str(i),str(E0)] #把文件号和E0放入同一个列表，方便放入csv的同一行
    with open(path2+'/id_prop.csv', 'a', encoding='utf-8') as f:
        f.write(','.join(csv_data)+'\n') #以','作为分割，把数据放入csv

In [19]:
# 把文件号和suface_strain写入CSV，创建ML命令CSV文件
def write_csv_strain(i,y,path2):
    csv_data = [str(i),str(y)] #把文件号和E0放入同一个列表，方便放入csv的同一行
    with open(path3+'/id_prop.csv', 'a', encoding='utf-8') as f:
        f.write(','.join(csv_data)+'\n') #以','作为分割，把数据放入csv

In [20]:
# 只能用于遍历具有相同命名规则且文件号具有顺序的文件
def main(n,path1,path2,path3,path4,data_energy):
    df = pd.DataFrame() #初始化一个dataframe，用于放置所有CONTCAR文件的晶格常数
    x = [] #用于储存大于200M文件夹的文件号
    y = [] #存储表面应力
    makedir(path1,path2,path3,path4) #创建放置CONTCAR文件和cif文件的文件夹
    for i in range(n): #计算每个符合的CONTCAR文件的晶格常数的均值
        dirpath = f'results{i}' #用于判断大小的文件夹路径
        a = get_origin_data(i,dirpath) #取出判断值a和文件夹路径
        if a == 1: #a等于1则表示该路径的文件夹大小大于200M
            x.append(i) #把大于200M文件夹的文件号放入x
            file = dirpath + '/CONTCAR' #file路径为大于200M文件夹下的CONTCAR文件
            data = read_file(file) #读取file并放进pandas
            df,dt = get_lattice(data,df,i)  #把所有适合的CONTCAR文件的晶格常数放入df
            a1,b1,c1 = get_abc(file,path4,i)
            suface_strain = get_strain_via_ptcu3(float(a1),float(b1),float(c1))
            y.append(suface_strain)
    mean = get_mean_lattice(df) #用df计算晶格常数均值
    a,b,c = get_mean_abc(mean) #取出三行晶格常数均值 
    print (x) #显示用于计算的文件号
    #读取原子坐标
    coordinate = pd.read_csv('POSCAR').iloc[7:39,:] #提取初始坐标
    for i in x: #从x取出大于200M文件夹的号数
        file = f'results{i}'
        print (file) #用于显示程序写入到哪个文件夹，展示进度用
        data = read_file(f'{file}/CONTCAR') #读取CONTCAR文件
        data = get_train_contcar(a,b,c,data) #替换CONTCAR文件中的晶格常数
        data.iloc[7:39,:] = coordinate #初始化原子坐标
        atom,counts = get_train_atom(data)
        last_line = get_oszicar(f'{file}/OSZICAR') #取出OSZICAR文件E0所在的最后一行
        E0 = get_e0(last_line,atom,counts,data_energy) #取出E0的值
        write_contcar(i,data,path1) #把新的CONTCAR文件放进path1
        os.system('ase convert {}/CONTCAR{} {}/{}.cif'.format(path1,i,path2,i)) #把CONTCAT文件转换成cif文件并放入path2
        write_csv_e0(i,str(E0),path2) #把文件号的E0放入csv，并放在path2
        write_csv_strain(i,y.pop(0),path3)

In [ ]:
n = 650
path1 = r'results_after' #放置CONTCAR文件
path2 = r'results_cif' #放置cif文件和csv文件
path3 = r'results_suface'
path4 = r'results_initial_cif'
data_energy = pd.DataFrame([-6.0982,-9.1198,-8.9174,-7.03679,-5.4686,-3.72838,
                               -2.2218,-8.2377852,-2.906231,-2.55672,-10.932452],
                              index = ['Pt','Cr','Mn','Co','Ni','Cu','Zn','Fe','Ga','In','Mo'])
main(n,path1,path2,path3,path4,data_energy) #main（文件夹个数，path1，path2,path3）

1 535691016


/home/gengzi/apps/miniconda3/envs/vasp_slurm/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


b'Skipping line 16: expected 1 fields, saw 3\n'


2 490223020


b'Skipping line 16: expected 1 fields, saw 3\n'


3 31116282
4 519460263


...

b'Skipping line 16: expected 1 fields, saw 3\n'


651 601911367


b'Skipping line 16: expected 1 fields, saw 3\n'


均值为：0    7.564701
1    0.002213
2   -0.003490
3    0.002166
4    7.586606
5   -0.001729
6   -0.003415
7   -0.001808
8    7.548890
Name: 均值, dtype: float64
[1, 2, 4, 5, 7, 8, 9, 11, 12, 16, 18, 20, 21, 23, 24, 27, 28, 29, 38, 39, 40, 41, 45, 47, 48, 50, 51, 53, 54, 55, 57, 58, 62, 66, 69, 70, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 93, 94, 96, 98, 99, 101, 102, 103, 105, 106, 107, 108, 109, 112, 113, 117, 118, 120, 121, 123, 124, 125, 126, 127, 129, 130, 135, 136, 138, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 178, 179, 180, 183, 184, 185, 186, 187, 188, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 207, 208, 210, 211, 213, 214, 215, 216, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 237, 239, 240, 241, 243, 244, 245, 247, 248, 249, 250, 254, 255, 256, 257, 259, 261, 264, 265, 266, 267

../results/POSCAR333
../results/POSCAR334
../results/POSCAR335
../results/POSCAR336
../results/POSCAR337
../results/POSCAR338
../results/POSCAR339
../results/POSCAR340
../results/POSCAR341
../results/POSCAR342
../results/POSCAR343
../results/POSCAR344
../results/POSCAR345
../results/POSCAR346
../results/POSCAR347
../results/POSCAR348
../results/POSCAR349
../results/POSCAR350
../results/POSCAR351
../results/POSCAR352
../results/POSCAR353
../results/POSCAR354
../results/POSCAR355
../results/POSCAR356
../results/POSCAR357
../results/POSCAR358
../results/POSCAR359
../results/POSCAR360
../results/POSCAR361
../results/POSCAR362
../results/POSCAR363
../results/POSCAR364
../results/POSCAR365
../results/POSCAR366
../results/POSCAR367
../results/POSCAR368
../results/POSCAR369
../results/POSCAR370
../results/POSCAR371
../results/POSCAR372
../results/POSCAR373
../results/POSCAR374
../results/POSCAR375
../results/POSCAR376
../results/POSCAR377
../results/POSCAR378
../results/POSCAR379
../results/PO